#### GROMACS for production
(See also: [`GROMACS_for_CHARMM-GUI.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/GROMACS_for_CHARMM-GUI.ipynb).)

#### Documentation
Please click ***↳ cells hidden*** below to show the documentation for this notebook.

##### About this software

> This notebook processes a **CHARMM-GUI system archive** (`.tgz`), producing a **GROMACS-ready folder for production runs**.
>
> A protein system prepared with the CHARMM-GUI **Solution Builder** or **Membrane Builder** must be provided. Optionally, a docked ligand conformation prepared with **Ligand Reader** may be provided, in which case the two structures and topologies will be merged into a protein-ligand complex. If the system spans a membrane, $z$-axis walls may be defined if desired.
>
> The recommended **minimisation** and **equilibration** simulations are then run with **GROMACS**, which automatically utilises the GPU if one is allocated. The equilibrated system is saved for a later production simulation. 

##### License

> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.

#### Configuration

In [ ]:
import os
import re
import shutil

#@markdown Specify the location of the **GROMACS project folder** to simulate. It should contain at least the input files `conf.gro` and `topol.top` (and probably `toppar/`), as well as a simulation parameters file `grompp.mdp` with any secondary inputs it references such as `index.ndx` or `restraint.gro`.
project_folder = "{GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid" #@param {type: "string"}
# default: {GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid

#@markdown Choose for how long the simulation should run.
simulation_duration_ns = 10 #@param {type: "integer"}
# default: 10

#@markdown Optionally, a group may be specified for which the RMSD from the initial conformation should be monitored -- the RMSD over time is plotted at the end of the simulation. Furthermore, if a threshold RMSD (in Angstroms) is exceeded, the simulation stops immediately. 
rmsd_group = "" #@param {type: "string"}
rmsd_early_stop_threshold_A = 12.0 #@param {type: "number"}
# default: 12.0

#@markdown Provide a unique filename prefix for this simulation. 
output_prefix = "sim" #@param {type: "string"}
# default: sim

#@markdown **After filling in this form, run the notebook by clicking *Runtime -> Run all* in the toolbar.**


#
# Make sure that the notebook is in the start directory
#

if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}


#
# Validate the input values
#


def google_drive_format(folder):
  if "{GoogleDrive}" in folder:
    if not folder.startswith("{GoogleDrive}"):
      raise ValueError(f"Error: {{GoogleDrive}} is a path prefix, but appears later: {folder}")
    if not os.path.isdir("/content/drive/MyDrive"):
      from google.colab import drive
      drive.mount("/content/drive")
  return folder.format(GoogleDrive="/content/drive/MyDrive")
  #             ^^^ raises KeyError if any {...} placeholder is present except {GoogleDrive}


# project_folder

project_folder = os.path.abspath(google_drive_format(project_folder.strip()))


# simulation_duration_ns

if simulation_duration_ns <= 0:
  raise RuntimeError(f"Error: simulation duration must be more than 0 ns, but got: {simulation_duration_ns} ns")

simulation_duration_ps = 1000 * simulation_duration_ns


# rmsd_group


# rmsd_early_stop_threshold

if rmsd_early_stop_threshold_A is None:
  rmsd_early_stop_threshold_A = 0.0

rmsd_early_stop_threshold_nm = rmsd_early_stop_threshold_A / 10.


# output_prefix

if not output_prefix:
  raise RuntimeError("Error: an output prefix must be provided")

if not re.match(r"^[0-9a-zA-Z_-]+$", output_prefix):
  raise RuntimeError(f"Error: special characters are not allowed in output prefix, but got: {output_prefix}")


#
# Use a clean scratch directory for the rest of the run
#

try:
  shutil.rmtree("scratch")
except FileNotFoundError:
  pass
shutil.copytree(project_folder, "scratch")
%cd "scratch"

#### Installation

In [ ]:
%%bash
#@markdown When this cell runs, **GROMACS** is loaded from Google Drive. If not found, it is downloaded and compiled from source code. (This takes a while.)

if [[ -d "/usr/local/gromacs" ]]; then
  exit 0  # already installed
fi

gromacs_version="gromacs-2023-rc1" #@param {type: "string"}
usr_local_gromacs="/content/drive/MyDrive/usr_local_${gromacs_version}.tar.gz"

export DEBIAN_FRONTEND=noninteractive
add-apt-repository -y "ppa:ubuntu-toolchain-r/test"
apt-get update
apt-get -y install --no-install-recommends "g++-9"
update-alternatives --install "/usr/bin/gcc" "gcc" "/usr/bin/gcc-9" 99
update-alternatives --install "/usr/bin/g++" "g++" "/usr/bin/g++-9" 99

if [[ -s "${usr_local_gromacs}" ]]; then
  tar -xzf "${usr_local_gromacs}" -C "/usr/local"
else
  wget "ftp://ftp.gromacs.org/gromacs/${gromacs_version}.tar.gz"
  tar -xzf "${gromacs_version}.tar.gz"
  cd "${gromacs_version}"
  mkdir "build"
  cd "build"
  cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA
  make -j $(nproc)
  make install # -> /usr/local/gromacs

  if [[ -d "$(dirname "$usr_local_gromacs")" ]]; then
    tar -czf "usr_local_gromacs.tar.gz" -C "/usr/local" "gromacs"
    mv "usr_local_gromacs.tar.gz" "${usr_local_gromacs}"
  fi
fi

#### Simulation

In [ ]:
%%writefile "run.bash"
project_folder="$1"
simulation_duration_ps=$2
rmsd_group="$3"
rmsd_early_stop_threshold_nm=$4
output_prefix="$5"
#@markdown Create a script to run the production simulation.

source "/usr/local/gromacs/bin/GMXRC.bash"
export GMX_MAXCONSTRWARN=-1

echo "Notice: saving output to folder: $project_folder"
sleep 1

block_ps=1000

{
  echo "***"
  date "+%F %T"
  echo "$(nproc) cores, $(free -m | awk 'NR == 2 { print $2 }') MiB"
  nvidia_smi="$(nvidia-smi --query-gpu="name,memory.total" --format="csv,noheader")"
  if (( $? == 0 )); then
    echo "$nvidia_smi"
  fi
  echo "***"
} | tee -a "$output_prefix.clock.log"

if [[ ! -s "$output_prefix.tpr" ]]; then
  # Ensure the first step is only $block_ps
  sim_dt=$(awk '$1 == "dt" { print $3 }' "grompp.mdp")
  steps_per_block=$(perl -e "printf(\"%.0f\n\", $block_ps / $sim_dt)")
  sed -i"" -E "s/^(nsteps *) = [0-9]+\$/\1 = $steps_per_block/" "grompp.mdp"

  # Construct the grompp command
  cmd=( gmx grompp -f "grompp.mdp" -o "$output_prefix.tpr" -c "conf.gro" -p "topol.top" -maxwarn 999 )
  if [[ -s "restraint.gro" ]]; then
    cmd+=( -r "restraint.gro" )
  fi
  if [[ -s "index.ndx" ]]; then
    cmd+=( -n "index.ndx" )
  fi
  "${cmd[@]}"
fi

# First block of 1 ns
if [[ ! -s "$output_prefix.trr" ]]; then
  echo "***"
  echo "Block: $block_ps ps / $simulation_duration_ps ps    (Block size: $block_ps ps)"
  echo "***"
  gmx mdrun -v -stepout 1000 -deffnm "$output_prefix"
fi

[[ -s "$output_prefix.xtc" ]] && trrext="xtc" || trrext="trr"

# Subsequent blocks of 1 ns until duration reached
t=$(gmx check -f "$output_prefix.$trrext" 2>&1 | tr "\r" "\n" | fgrep "Last frame" | awk '{ print $5 }')
t=$(perl -le "printf(\"%.0f\n\", $t)")

while true; do
  t=$(($t + $block_ps))

  grep "ns/day" "$output_prefix.log" -A2 -B2 >> "$output_prefix.clock"

  # Send output to the output folder
  for f in $output_prefix.*; do
    if [[ "${f: -1}" == "~" || ! -s "$f" ]]; then
      continue
    fi
    if [[ -s "$project_folder/$f" ]]; then
      mv "$project_folder/$f" "$project_folder/$f~"
    fi
    cp "$f" "$project_folder/"
  done
  
  # Check RMSD of specified group
  if [[ ! -z "$rmsd_group" && ! -z "$rmsd_early_stop_threshold_nm" ]] && perl -le "$rmsd_early_stop_threshold_nm > 0.001 ? exit 0 : exit 1"; then
    {
      echo "C-alpha"
      echo "$rmsd_group"
    } | gmx rms -s "$output_prefix.tpr" -f "$output_prefix.$trrext" -b $(($t - $block_ps))
    rmsd_alarm=$(sed '/^[#@&]/d; /^ *$/d' "rmsd.xvg" | awk "\$2 > $rmsd_early_stop_threshold_nm" | wc -l)
    if (( $rmsd_alarm > 0 )); then
      echo "Notice: detected that RMSD of $rmsd_group exceeded threshold $rmsd_early_stop_threshold_nm nm" >&2
      break
    fi
  fi

  # End if we have reached the end of the simulation duration
  if (( $t >= $simulation_duration_ps )); then
    break
  fi

  # Run the simulation
  echo "***"
  echo "Block: $t ps / $simulation_duration_ps ps    (Block size: $block_ps ps)"
  echo "***"
  gmx convert-tpr -s "$output_prefix.tpr" -extend $block_ps -o "tprout.tpr"
  mv "tprout.tpr" "$output_prefix.tpr"
  gmx mdrun -s "$output_prefix.tpr" -cpi "$output_prefix.cpt" -v -stepout 1000 -deffnm "$output_prefix"
done

gmx trjconv -f "$output_prefix.$trrext" -s "$output_prefix.tpr" -pbc whole -o "$output_prefix.whole.xtc" <<< 0
cp "$output_prefix.whole.xtc" "$project_folder/"

if [[ ! -z "$rmsd_group" ]]; then
  {
    echo "C-alpha"
    echo "$rmsd_group"
  } | gmx rms -s "$output_prefix.tpr" -f "$output_prefix.$trrext"
  echo "Time_(ns),RMSD_(A)" > "$output_prefix.rmsd.csv"
  sed '/^[#@&]/d; /^ *$/d' "rmsd.xvg" | awk '{ print $1 / 1000","10 * $2 }' >> "$output_prefix.rmsd.csv"
fi

In [ ]:
#@markdown Execute the simulation script:
#@markdown 1. Run a loop of 1 ns blocks until the **production simulation** is complete -- each loop saves a partial output.
#@markdown 2. Save the postprocessed final output.
!bash "run.bash" "$project_folder" "$simulation_duration_ps" "$rmsd_group" "$rmsd_early_stop_threshold_nm" "$output_prefix"
!sleep 10

In [ ]:
#@markdown If a group was monitored for its RMSD along its simulated trajectory, save a plot of its RMSD over time.

if os.path.isfile(f"{output_prefix}.rmsd.csv"):
  import pandas as pd
  import matplotlib.pyplot as plt
  import seaborn as sns
  data = pd.read_csv(f"{output_prefix}.rmsd.csv")
  lp = sns.lineplot(data=data, x="Time_(ns)", y="RMSD_(A)")
  lp.figure.savefig(f"{output_prefix}.rmsd.png")
  plt.show()

  shutil.copy(f"{output_prefix}.rmsd.csv", project_folder)
  shutil.copy(f"{output_prefix}.rmsd.png", project_folder)

In [ ]:
#@markdown Finally, disconnect the runtime. (This option is ignored if the project folder is not in your Google Drive.)
disconnect = True #@param {type: "boolean"}

if disconnect and project_folder.startswith("/content/drive/MyDrive/"):
  from google.colab import runtime
  runtime.unassign()